Create view between transcript and context tables

In [10]:
import pandas as pd
import sqlite3

def create_transcript_context_view(new_view_name, transcript_table, context_table):
    write_conn = sqlite3.connect("../automated_metrics.db")
    write_conn.execute("PRAGMA foreign_keys = 1")
    cur = write_conn.cursor()
    
    create_view_sql = f"""
    CREATE VIEW IF NOT EXISTS {new_view_name} AS
    SELECT transcript.transcript_id as transcript_id,
        {transcript_table}.case_facts as case_facts,
        {transcript_table}.legal_question as legal_question,
        {context_table}.context_id as context_id,
        {context_table}.context as context,
        {context_table}.justice as justice,
        {context_table}.turn_text as turn_text
    FROM {transcript_table}
    JOIN {context_table} ON {transcript_table}.transcript_id = {context_table}.transcript_id;
    """

    cur.execute(create_view_sql)
    write_conn.commit()
    write_conn.close()

create_transcript_context_view("transcript_and_context", "transcript", "context")

Make sampled context table

In [4]:
import pandas as pd
import sqlite3
write_conn = sqlite3.connect("../automated_metrics.db")
write_conn.execute("PRAGMA foreign_keys = 1")
cur = write_conn.cursor()

cur.execute("CREATE TABLE IF NOT EXISTS hundred_sampled_context(context_id TEXT PRIMARY KEY, " \
            "context TEXT, " \
            "justice TEXT, " \
            "turn_text TEXT, " \
            "transcript_id TEXT, " \
            "FOREIGN KEY (transcript_id) REFERENCES transcript(transcript_id) ON DELETE CASCADE);")

df = pd.read_sql_query("SELECT * FROM context;", write_conn)
sampled_elements = df.sample(n=100)
sampled_elements.to_sql('hundred_sampled_context', write_conn, if_exists='append', index=False)
write_conn.close()

Make view for all transcripts and the sampled contexts

In [11]:
create_transcript_context_view("hundred_sampled_transcript_and_context", "transcript", "hundred_sampled_context")

Create view for metrics jobs

In [3]:
write_conn = sqlite3.connect("../automated_metrics.db")
write_conn.execute("PRAGMA foreign_keys = ON;")
cur = write_conn.cursor()

df = pd.read_sql_query("SELECT * FROM remark;", write_conn)
remark_table = "remark"
context_table = "context"
transcript_table = "transcript"
create_view_sql = f"""CREATE VIEW IF NOT EXISTS remark_transcript_context AS
SELECT
    {remark_table}.remark_id as remark_id,
    {remark_table}.justice as justice,
    {remark_table}.remark_text as remark_text,
    {remark_table}.log_id as remark_log_id,
    {context_table}.context as context,
    {transcript_table}.case_facts as case_facts,
    {transcript_table}.legal_question as legal_question
FROM remark
JOIN context ON remark.context_id = context.context_id
JOIN transcript ON context.transcript_id = transcript.transcript_id;
"""
cur.execute(create_view_sql)
write_conn.commit()
write_conn.close()

In [16]:
write_conn = sqlite3.connect("../automated_metrics.db")
write_conn.execute("PRAGMA foreign_keys = ON;")
cur = write_conn.cursor()

df = pd.read_sql_query("SELECT * FROM remark;", write_conn)

In [15]:
remark_table = "remark"
context_table = "context"
transcript_table = "transcript"
create_view_sql = f"""CREATE VIEW IF NOT EXISTS remark_transcript_context AS
SELECT
    {remark_table}.remark_id as remark_id,
    {remark_table}.justice as justice,
    {remark_table}.remark_text as remark_text,
    {remark_table}.log_id as remark_log_id,
    {context_table}.context as context,
    {transcript_table}.case_facts as case_facts,
    {transcript_table}.legal_question as legal_question
FROM remark
JOIN context ON remark.context_id = context.context_id
JOIN transcript ON context.transcript_id = transcript.transcript_id;
"""

create_view_sql

'CREATE VIEW IF NOT EXISTS remark_transcript_context AS\nSELECT\n    remark.remark_id as remark_id,\n    remark.justice as justice,\n    remark.remark_text as remark_text,\n    context.context as context,\n    transcript.case_facts as case_facts,\n    transcript.legal_question as legal_question\nFROM remark\nJOIN context ON remark.context_id = context.context_id\nJOIN transcript ON context.transcript_id = transcript.transcript_id;\n'